# Military Drone Strikes, Afgan, Pakistan, Yemen and Somalia (Part 2)

This script is a continuation of Part 1 where we take our clensed data in the drone-strikes.xlsx spreadsheet and plot its data.

Plotting is carried out using the Plotly graphing library


In [ ]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import plotly.express as px
import plotly.graph_objects as go
import re
import mitosheet

# import our config file which hol;ds our mapbox access token
# we have the config.py file in our .gitignore so the file is 
# not synced to git thereby exposing our key
from config import token

## Lets get plotting our data

### Read in our processed data file from part 1

In [ ]:
df = pd.read_excel('drone-strikes.xlsx')

### Create a template that we can use for our Charts
To save time and cut down on repetitiveness it is good to define a template of our reguarly used Plotly attributes. 

In [ ]:
# Create our custom template
da_template = {
    "layout": go.Layout(
        font={
            "family": "Arial",
            "size": 12,
            "color": "#707070",
        },
        title={
            "font": {
                "family": "Arial",
                "size": 18,
                "color": "#1f1f1f",
            },
        },
        height=800,
        width=800,
        paper_bgcolor ='rgba(0,0,0,0)',
        plot_bgcolor ='rgba(0,0,0,0)',
        colorway = px.colors.qualitative.G10,
        title_x = 0.01
    )
}

In [ ]:
# Function to style a title and subtitle. we use this to credit our plots with data sources
# Inspiration from https://towardsdatascience.com/a-clean-style-for-plotly-charts-250ba2f5f015
def format_title(title, subtitle=None, subtitle_font_size=14):
    title = f'<b>{title}</b>'
    if not subtitle:
        return title
    subtitle = f'<span style="font-size: {subtitle_font_size}px;">{subtitle}</span>'
    return f'{title}<br>{subtitle}'

### Chart utilising Plotly scatter_mapbox
This chart makes use of the Latitude and Longitude data from our cleansed dataframe
You need to have a mapbox access token to access their maps. Failure to have a token will result in a blank chart

In [ ]:
# Import our Mapbox access token from our config file
mapbox_token = token

px.set_mapbox_access_token(mapbox_token)
fig = px.scatter_mapbox(df,
                        lat='Latitude',
                        lon='Longitude',
                        color='Maximum total people killed',
                        size='Maximum total people killed',
                        # Defining our colour scale for mapping the strikes
                        # https://plotly.com/python/builtin-colorscales/
                        color_continuous_scale=px.colors.cyclical.IceFire,
                        size_max=20,
                        width=800, 
                        height=800,
                        title=format_title("US Drone Strikes 2002 - 2018", 
                                           "Data collected from https://dronewars.github.io/data/"),
                        )

# Set the center point of the map and the starting zoom 
fig.update_mapboxes(center=dict(lat = 21, lon = 50),
                    zoom = 3,
                   )

fig.update_traces(
    # This removed the colorbar
    marker_coloraxis=None
)

fig.show()

### How many Strikes were carried out by the sitting President of the United States

In [ ]:
dfprez = df.groupby('President')
dfprez = dfprez.apply(lambda x: x['Maximum total people killed'].count())

# Build our graph
fig = px.bar(dfprez,
             x=dfprez.index,
             y=dfprez.values,
             title=format_title("Total Number of Missions flown under each US Presidents time in Office starting in 2002<br> up to and including 2018", 
                                "Data collected from https://dronewars.github.io/data/"),
             template=da_template
             )

# Set angle of text on the x axis
fig.update_xaxes(title_text='<b>Sitting President<b>',
                 showline=True,
                 linewidth=1,
                 linecolor='#071633'
                )

# Set Y axis formating 
fig.update_yaxes(title_text='<b>Number of Strikes<b>',
                )

fig.update_layout(height=800,
                  #width=800
                  )

fig.update_traces(text= dfprez.values,
                  textposition='inside'
                  )

### How many many missions were flown each year?

In [ ]:
dfyear = df.groupby('Year')
dfyear = dfyear.apply(lambda x: x['Year'].count())

# Build our graph
fig = px.bar(dfyear,
             x=dfyear.index,
             y=dfyear.values,
             title=format_title("Total Number of Missions Flown Each Year", 
                                "Data collected from https://dronewars.github.io/data/"),
             template=da_template
             )

fig.update_layout(height=800,
                  width=800
                  )

# Set angle of text on the x axis
fig.update_xaxes(title_text='<b>Year<b>',
                 tickangle=45,
                 showline=True,
                 linewidth=1,
                 linecolor='#071633'
                )

# Set Y axis formating 
fig.update_yaxes(title_text='<b>Number of Strikes<b>',
                )

fig.update_traces(text= dfyear.values,
                  textposition='outside'
                  )

## Drone Strikes Distributed by Day of Week

In [ ]:
# Setup our data for defining a Category order
day_order = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
cat_type = CategoricalDtype(categories=day_order, ordered=True)

In [ ]:
# Set our 'Day' Column to our Categorical Type
df['Day'] = df['Day'].astype(cat_type)

In [ ]:
# Get Number of Drone Strikes grouped by Day of week
dfday = df.groupby('Day')
dfday = dfday.apply(lambda x: x['Day'].count())

In [ ]:
# Get the mean and median of the strikes
mean_strike = dfday.mean()

### Demo Graph Showing number of Drone Strikes by day

In [ ]:
# Build our graph
fig = px.bar(dfday, x=dfday.index, y=dfday.values,
             title=format_title("Number of Drone Strikes by Day",
                                "Data collected from https://dronewars.github.io/data/"),
             template=da_template
             )

# Add a dashed horizontal line to show our mean
fig.add_hline(y=mean_strike, line_dash = 'dash', line_color = 'Red')

# Add a rectangle bounding the area on the Y axis from 220 - 260 strikes a day
# Set it to the colour green and an opacity of 0.3.
fig.add_hrect(y0=212, y1=272, line_width=0, fillcolor="green", opacity=0.3)

fig.update_layout(height=800                  
                 )

# Set angle of text on the x axis
fig.update_xaxes(title_text='<b>Day<b>',
                 tickangle=45,
                 showline=True, 
                 linewidth=1,
                 linecolor='#071633'
                )

# Set Y axis formating 
fig.update_yaxes(title_text='<b>Number of Strikes<b>',
                 # Displays y axis line, sets the width and colour 
                 #showline=True,
                 #linewidth=1,
                 #linecolor='#071633'
                )

fig.update_traces(text = dfday.values,
                  # Set text to show outside the bars
                  textposition='outside',
                  
                 # Set bar colour to a specific color
                  marker_color='Blue'
                 )

# Add annotations
# Add a description to our Mean line
fig.add_annotation(x='Tuesday', y=246,
            text="<b>Mean (242)</b>",
            # Do not show an arrow
            showarrow=False,
            arrowhead=1,
            font=dict(
                color="Red",
                size=12)
                  )

# Add a description to our green rectangle
# with a text annotation and an arrow
fig.add_annotation(x='Monday', y=211,
            text="<b>Lower Limit</b>",
            # Show an arrow
            showarrow=True,
            arrowhead=1,
            # Set annotation text 
            ax=10,
            ay=30,       
            arrowcolor='White',
            opacity=1.0,
            font=dict(
                color="cyan",
                size=12)
                  )

# Add a description to our green rectangle
# with a text annotation and an arrow
fig.add_annotation(x='Saturday', y=272,
            text="<b>Upper Limit</b>",
            # Dont show an arrow
            showarrow=True,
            arrowhead=1,
            font=dict(
                color="Cyan",
                size=12)
                  )

# Hide colour Scale
fig.update_coloraxes(showscale=False)

fig.show()